In [1]:
import pandas as pd
import os.path
in_filename = r'data' + os.path.sep + r'raw_beer_data.csv'
out_filename = r'data' + os.path.sep + r'final_beer_data.csv'

# Part 1: Data Munging

First, I'll read the data from the downloaded .csv file and clean it up to help make analysis easier later. First thing will be to only select the relevant columns from the dataframe

In [2]:
# read in data to dataframe
in_path = os.path.join(os.getcwd(), in_filename)
df = pd.read_csv(in_path, parse_dates=['created_at'])

In [3]:
df.head(3)

,beer_name,brewery_name,beer_type,beer_abv,beer_ibu,comment,venue_name,venue_city,venue_state,venue_country,...,created_at,checkin_url,beer_url,brewery_url,brewery_country,brewery_city,brewery_state,flavor_profiles,purchase_venue,serving_type
0,Truth,Rhinegeist Brewery,IPA - American,7.2,75,very hoppy,Rhinegeist Brewery,Cincinnati,OH,United States,...,2015-10-06 20:04:06,https://untappd.com/c/232541746,https://untappd.com/beer/393430,https://untappd.com/brewery/69953,United States,Cincinnati,OH,NaN,NaN,NaN
1,Franz,Rhinegeist Brewery,Märzen,5.4,20,NaN,NaN,NaN,NaN,NaN,...,2015-10-06 20:04:24,https://untappd.com/c/232541985,https://untappd.com/beer/454961,https://untappd.com/brewery/69953,United States,Cincinnati,OH,NaN,NaN,NaN
2,Hugh,Rhinegeist Brewery,Hefeweizen,5.4,10,Good hefe. Cloves and bananas,Rhinegeist Brewery,Cincinnati,OH,United States,...,2015-10-06 20:05:38,https://untappd.com/c/232542512,https://untappd.com/beer/1230095,https://untappd.com/brewery/69953,United States,Cincinnati,OH,NaN,NaN,NaN


In [4]:
# let's see all the columns of data we're offered
df.columns

Index(['beer_name', 'brewery_name', 'beer_type', 'beer_abv', 'beer_ibu',
       'comment', 'venue_name', 'venue_city', 'venue_state', 'venue_country',
       'venue_lat', 'venue_lng', 'rating_score', 'created_at', 'checkin_url',
       'beer_url', 'brewery_url', 'brewery_country', 'brewery_city',
       'brewery_state', 'flavor_profiles', 'purchase_venue', 'serving_type'],
      dtype='object')

In [5]:
# lots of those columns I won't be using, so I'll get rid of them now to make the df easier to use
cols_to_del = ['comment', 'checkin_url', 'beer_url', 'brewery_url', 'flavor_profiles', 'purchase_venue', 'serving_type']
df = df.drop(cols_to_del, axis=1)
df.head(3)

,beer_name,brewery_name,beer_type,beer_abv,beer_ibu,venue_name,venue_city,venue_state,venue_country,venue_lat,venue_lng,rating_score,created_at,brewery_country,brewery_city,brewery_state
0,Truth,Rhinegeist Brewery,IPA - American,7.2,75,Rhinegeist Brewery,Cincinnati,OH,United States,39.1172,-84.5201,3.00,2015-10-06 20:04:06,United States,Cincinnati,OH
1,Franz,Rhinegeist Brewery,Märzen,5.4,20,NaN,NaN,NaN,NaN,NaN,NaN,3.25,2015-10-06 20:04:24,United States,Cincinnati,OH
2,Hugh,Rhinegeist Brewery,Hefeweizen,5.4,10,Rhinegeist Brewery,Cincinnati,OH,United States,39.1172,-84.5201,3.50,2015-10-06 20:05:38,United States,Cincinnati,OH


I next noticed some beers have null values for brewery_state and brewery_city. I'm going to put them in manually - I'll google the cities and add them. For non-American beers, I'll treat the state differently depending on the country.

In [6]:
# this fixes all the beers that had a null value for city
df.loc[df.brewery_name.str.startswith('Blank Slate'), 'brewery_city'] = 'Cincinnati'
df.loc[df.brewery_name.str.startswith('Trzech'), 'brewery_city'] = 'Tarnow'
df.loc[df.brewery_name.str.startswith('Bang Bang'), 'brewery_city'] = 'Madrid'
df.loc[df.brewery_name.str.startswith('CCVK'), 'brewery_city'] = 'Madrid'
df.loc[df.brewery_name.str.startswith('Gabarrera'), 'brewery_city'] = 'Mataelpino'

In [7]:
# this fixes all the beers that had a null value for state, as well as a few cleanup issues
df.loc[df.brewery_city.str.contains('Liège'),'brewery_city'] = 'Liège'
df.loc[df.brewery_city.str.contains('Liège'),'brewery_state'] = 'Liège'
df.loc[df.brewery_city.str.contains('Cincinnati'),'brewery_state'] = 'OH'
df.loc[df.brewery_city.str.contains('Cincinnati'),'brewery_state'] = 'OH'
df.loc[df.brewery_city.str.contains('Madrid'), 'brewery_state'] = 'Madrid'
df.loc[df.brewery_city.str.contains('København'), 'brewery_state'] = 'Region Hovedstaden'
df.loc[df.brewery_city.str.contains('Copenhagen'), 'brewery_state'] = 'Region Hovedstaden'
df.loc[df.brewery_city.str.contains('Mataelpino'), 'brewery_state'] = 'Madrid'
df.loc[df.brewery_city.str.contains('København'), 'brewery_state'] = 'Region Hovedstaden'
df['brewery_state'].fillna(df['brewery_city'], inplace=True)

## Removing Duplicates

I'm going to remove any beer that appears more than once. If you're interested in my reasoning:
     
1. I use Untappd primarily to keep a record of `1. Which beers I drink` and `2. What I thought of those beers`. I don't use untappd to keep track of *which beers I drink the most* and have been inconsistent at best with duplicates. 
2. I've never defined a set rule on duplicates; if I buy a 6-pack, for example, do I check the beer in once or six times? 
3. Many duplicates are beers I drank without realizing I'd had before. Duplicates I intentionally checked in multiple times are usually my favorite beers, while I don't check in "garbage beers" that I have much more regularly than craft beers (i.e. at a sporting event or a night out with friends)
4. Much of the analysis I'm interested in is about particular beer styles and breweries, and if a beer has more than one check it'll skew the numbers unfairly. Each beer should be worth the same as any other beer.

In [8]:
# comment out if you want to keep duplicates
df.drop_duplicates('beer_name', keep='last', inplace=True)

Note that I kept the *last* occurance of a beer and not the first, because I figured the most accurate rating of a beer would be the most recent, i.e. the rating that the beer would currently have according to my profile.

## Simplifying Styles

I want to make a new column called 'simplified_style' which combines substyles into one category, i.e. all variants of IPA into one IPA group. I'll mainly do this with beers that differ only regional variety, such as American and English Brown Ales both being Brown Ales. This isn't the same as sorting into beer families, however. Also, some styles with low numbers (such as Pumpkin Beer, where I have 3 total split among substyles) I'll leave alone, as I'm only interested in the largest few styles.

Styles I'll simplfy:

* Blonde Ale
* Brown Ale
* IPA*
* Lager**
* Pale Ale
* Pilsner
* Porter
* Red Ale
* Sour
* Stout

\*For IPAs, I will follow the Brewer's Association Style Guide and put them into three categories: American IPA, English IPA, and Double/Imperial IPA. To any IPA buffs offended by this: sorry, but my data, my rules!

\**For lagers, I'm simply going to rewrite the format so that 'Lager - Euro' appears written as 'Euro Lager'

In [9]:
# create new column and move it next to current style
df['simple_style'] = df['beer_type']
cols = df.columns.tolist()
cols = cols[:3] + [cols[-1]] + cols[3:-1]
df = df[cols]

In [10]:
# rename styles
df.loc[df.simple_style.str.startswith('Blonde Ale'), 'simple_style'] = 'Blonde Ale'
df.loc[df.simple_style.str.startswith('Brown Ale'), 'simple_style'] = 'Brown Ale'
df.loc[df.simple_style.str.startswith('Pale Ale'), 'simple_style'] = 'Pale Ale'
df.loc[df.simple_style.str.startswith('Pilsner'), 'simple_style'] = 'Pilsner'
df.loc[df.simple_style.str.startswith('Porter'), 'simple_style'] = 'Porter'
df.loc[df.simple_style.str.startswith('Red Ale'), 'simple_style'] = 'Red Ale'
df.loc[df.simple_style.str.startswith('Sour'), 'simple_style'] = 'Sour'
df.loc[df.simple_style.str.startswith('Stout'), 'simple_style'] = 'Stout'

# now, for the IPAs
df.loc[df.simple_style.str.startswith('IPA - American'), 'simple_style'] = 'American IPA'
df.loc[df.simple_style.str.startswith('IPA - Session'), 'simple_style'] = 'American IPA'
df.loc[df.simple_style.str.startswith('IPA - Imperial'), 'simple_style'] = 'Imperial IPA'
df.loc[df.simple_style.str.startswith('IPA - White'), 'simple_style'] = 'American IPA'
df.loc[df.simple_style.str.startswith('IPA - Belgian'), 'simple_style'] = 'American IPA'
df.loc[df.simple_style.str.startswith('IPA - Black'), 'simple_style'] = 'American Black Ale'
df.loc[df.simple_style.str.startswith('IPA - Triple'), 'simple_style'] = 'Imperial IPA'
df.loc[df.simple_style.str.startswith('IPA - Red'), 'simple_style'] = 'American IPA'
df.loc[df.simple_style.str.startswith('IPA - New England'), 'simple_style'] = 'American IPA'
df.loc[df.simple_style.str.startswith('IPA - English'), 'simple_style'] = 'English IPA'
df.loc[df.simple_style.str.startswith('Rye IPA'), 'simple_style'] = 'American IPA'
df.loc[df.simple_style.str.startswith('IPA - International'), 'simple_style'] = 'American IPA'

## Adding new time columns

Later, I'm going to analyze how many beers I've drank per week and month, so I'm going to help myself out by creating new columns that represent the weeks and months of the year for each check-in. To attempt to standardize and set up future potential for rewriting code to work as a program for other users, I'm going to find the first week and set that as the 'base' and count up from there.

In [11]:
# find base week
base_year = df.created_at.dt.year.min()
base_week = (df.created_at.dt.weekofyear + (df.created_at.dt.year % base_year * 52)).min()

In [12]:
# create week column, subtract all by base week to normalize
df['week'] = ((df.created_at.dt.weekofyear + (df.created_at.dt.year % base_year * 52)) - base_week)

The month column will be a bit different; for the first year (`base_year`) the month numbers will just be 1-12, and then count up from there. So for example, I started using Untappd in October 2015. `2015` should be my base year, and my first entires should have a `month` value equal to `10`. Then, `13` would be January 2016, `25` would be January 2017, etc.

In [13]:
# create month column
df['month'] = (df.created_at.dt.month + (df.created_at.dt.year % base_year * 12))

## Output to new .csv file
Last step - output this updated dataframe to a new .csv file that can be accessed later for analysis!

In [14]:
# have to encode as utf-8, as ANSI was giving errors with foreign beers and unique characters
out_path = os.path.join(os.getcwd(), out_filename)
df.to_csv(out_path, index=False, encoding='utf-8')